In [1]:
#%%
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from psychopy import data
import os

# ------------------------------
# User settings
# ------------------------------
participant_num = "145"

input_csv = f"/Users/wouter/Documents/phd/projects/psychophysics/experiments/dipperV2/analysis/{participant_num}_baseline.csv"
output_dir = f"./Output/Exp3/Baseline/{participant_num}_plots/"
detection_prob = 0.5   # threshold

os.makedirs(output_dir, exist_ok=True)

# ------------------------------
# Load data
# ------------------------------
df = pd.read_csv(input_csv)

# Remove extremely long RTs
df = df[df["RT"] <= 90]

# ------------------------------
# Fit psychometric curves
# ------------------------------
unique_labels = df['label'].unique()
thresholds = {}

for label in unique_labels:
    cond_df = df[df['label'] == label]
    
    # Remove false positives for "_null" trials
    mask_false_pos = (cond_df['label'].str.endswith('_null')) & (cond_df['response'] == 1)
    cond_df = cond_df[~mask_false_pos]
    
    if len(cond_df) < 2:
        continue  # skip empty or tiny conditions
    
    # Fit logistic
    fit_result = data.FitLogistic(cond_df['TC'], cond_df['response'], expectedMin=0.0)
    
    # Get threshold
    threshold = fit_result.inverse(detection_prob)
    thresholds[label] = threshold
    
    # Plot
    smooth_x = np.linspace(cond_df['TC'].min()*0.8, cond_df['TC'].max()*1.2, 500)
    smooth_y = fit_result.eval(smooth_x)
    
    plt.figure(figsize=(5,4))
    plt.scatter(cond_df['TC'], cond_df['response'], color='black', label='Responses')
    plt.plot(smooth_x, smooth_y, color='red', label='Fit')
    plt.axhline(detection_prob, color='blue', linestyle='--', label=f'{detection_prob*100:.0f}% threshold')
    plt.title(label)
    plt.xlabel('Target Contrast (TC)')
    plt.ylabel('Detection Probability')
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"{output_dir}/{label}_psychometric.png")
    plt.close()

# ------------------------------
# Save thresholds to CSV
# ------------------------------
pd.DataFrame.from_dict(thresholds, orient='index', columns=['Threshold']).to_csv(f"{output_dir}/thresholds.csv")
print("Done!")

Done!
